In [1]:

import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:

data = pd.read_csv("jester-data-1.csv", header = None)
d = data.to_latex()

In [3]:
text_file = open("Output.txt", "w")
text_file.write(d)
text_file.close()

In [4]:
training = data[:int(len(data)*0.8)]
test = data[int(len(data)*0.8+1):int(len(data)*0.9)]
validation = data[int(len(data)*0.9+1):]

In [5]:

n_features = 2

user_ratings = training.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [6]:

def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 20):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(not rating == 99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if(iteration%10000 == 0 ):
        print mse

In [7]:

#train(0,0,10)
sgd()
#user_preferences 
#item_features

61.9477035447
32.3242881564
25.51330482
24.5459055918
24.4433285433
24.43095411
24.428655867
24.4277228293
24.4270678167
24.42651274
24.4260126427
24.4255478362
24.4251038739
24.42466602
24.4242162878
24.4237302921
24.423172804
24.4224910099
24.4216040867
24.4203869634


In [8]:

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[  7.476e+01,   8.928e-01,   2.865e-01, ...,   6.573e-01,
          7.011e-02,   1.125e+00],
       [  9.632e+01,   1.089e+00,   2.446e-01, ...,   8.106e-01,
          4.477e-02,   1.385e+00],
       [  5.452e+01,   7.012e-01,   3.103e-01, ...,   5.088e-01,
          8.823e-02,   8.721e-01],
       ..., 
       [  1.037e+02,   1.213e+00,   3.470e-01, ...,   8.969e-01,
          7.885e-02,   1.534e+00],
       [  5.996e+01,   7.139e-01,   2.252e-01, ...,   5.259e-01,
          5.457e-02,   8.997e-01],
       [  9.911e+01,   1.238e+00,   4.889e-01, ...,   9.031e-01,
          1.329e-01,   1.547e+00]])

In [9]:
values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns
comparison_data.applymap(lambda (x,y): "(%2.3f|%2.3f)"%(x,y))

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,(74.000|74.764),(-7.820|0.893),(8.790|0.286),(-9.660|0.332),(-8.160|-1.117),(-7.520|0.439),(-8.500|1.494),(-9.850|-0.337),(4.170|-0.346),(-8.980|-0.371),...,(2.820|1.651),(99.000|1.021),(99.000|2.009),(99.000|0.934),(99.000|0.960),(99.000|1.304),(-5.630|1.460),(99.000|0.657),(99.000|0.070),(99.000|1.125)
1,(100.000|96.325),(4.080|1.089),(-0.290|0.245),(6.360|0.411),(4.370|-1.590),(-2.380|0.617),(-9.660|1.921),(-0.730|-0.519),(-5.340|-0.474),(8.880|-0.588),...,(2.820|2.146),(-4.950|1.306),(-0.290|2.635),(7.860|1.348),(-0.190|1.084),(-2.140|1.664),(3.060|1.809),(0.340|0.811),(-4.320|0.045),(1.070|1.385)
2,(49.000|54.521),(99.000|0.701),(99.000|0.310),(99.000|0.256),(99.000|-0.692),(9.030|0.277),(9.270|1.093),(9.030|-0.176),(9.270|-0.230),(99.000|-0.181),...,(99.000|1.189),(99.000|0.752),(99.000|1.426),(9.080|0.563),(99.000|0.825),(99.000|0.964),(99.000|1.124),(99.000|0.509),(99.000|0.088),(99.000|0.872)
3,(48.000|50.007),(99.000|0.554),(8.350|0.104),(99.000|0.210),(99.000|-0.854),(1.800|0.330),(8.160|0.997),(-2.820|-0.285),(6.210|-0.251),(99.000|-0.326),...,(99.000|1.118),(99.000|0.676),(99.000|1.377),(0.530|0.727),(99.000|0.534),(99.000|0.861),(99.000|0.925),(99.000|0.414),(99.000|0.015),(99.000|0.707)
4,(91.000|93.810),(8.500|1.139),(4.610|0.397),(-4.170|0.422),(-5.390|-1.357),(1.360|0.534),(1.600|1.876),(7.040|-0.397),(4.610|-0.426),(-0.440|-0.433),...,(5.190|2.066),(5.580|1.284),(4.270|2.506),(5.190|1.128),(5.730|1.251),(1.550|1.642),(3.110|1.854),(6.550|0.836),(1.800|0.102),(1.600|1.430)
5,(100.000|91.632),(-6.170|1.028),(-3.540|0.216),(0.440|0.389),(-8.500|-1.533),(-7.090|0.594),(-4.320|1.827),(-8.690|-0.505),(-0.870|-0.455),(-6.650|-0.574),...,(-3.540|2.044),(-6.890|1.241),(-0.680|2.513),(-2.960|1.302),(-2.180|1.011),(-3.350|1.581),(0.050|1.711),(-9.080|0.766),(-5.050|0.037),(-3.450|1.309)
6,(47.000|50.271),(99.000|0.573),(99.000|0.136),(99.000|0.216),(99.000|-0.819),(8.590|0.319),(-9.850|1.003),(7.720|-0.265),(8.790|-0.246),(99.000|-0.299),...,(99.000|1.119),(99.000|0.682),(99.000|1.372),(99.000|0.694),(99.000|0.576),(2.330|0.870),(99.000|0.949),(99.000|0.426),(99.000|0.027),(99.000|0.727)
7,(100.000|95.940),(6.840|1.137),(3.160|0.351),(9.170|0.424),(-6.210|-1.454),(-8.160|0.570),(-1.700|1.917),(9.270|-0.444),(1.410|-0.448),(-5.190|-0.491),...,(7.230|2.121),(-1.120|1.308),(-0.100|2.584),(-5.680|1.218),(-3.160|1.211),(-3.350|1.672),(2.140|1.864),(-0.050|0.839),(1.310|0.084),(0.000|1.435)
8,(100.000|99.460),(-3.790|1.122),(-3.540|0.247),(-9.420|0.424),(-6.890|-1.648),(-8.740|0.640),(-0.290|1.984),(-5.290|-0.540),(-8.930|-0.491),(-7.860|-0.612),...,(4.370|2.217),(-0.290|1.348),(4.170|2.723),(-0.290|1.399),(-0.290|1.112),(-0.290|1.718),(-0.290|1.865),(-0.290|0.835),(-3.400|0.044),(-4.950|1.428)
9,(72.000|73.922),(3.010|0.924),(5.150|0.367),(5.150|0.340),(3.010|-1.004),(6.410|0.398),(5.150|1.480),(8.930|-0.276),(2.520|-0.324),(3.010|-0.293),...,(99.000|1.620),(4.470|1.015),(99.000|1.954),(99.000|0.826),(99.000|1.052),(99.000|1.301),(99.000|1.493),(99.000|0.674),(99.000|0.100),(99.000|1.155)


In [10]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 74.7640996456)","(-7.82, 0.892830472519)","(8.79, 0.286467444063)","(-9.66, 0.332326355228)","(-8.16, -1.11741854348)","(-7.52, 0.438526944946)","(-8.5, 1.49435908557)","(-9.85, -0.336674556688)","(4.17, -0.346462148091)","(-8.98, -0.371220798459)",...,"(2.82, 1.65108364776)","(99.0, 1.02067741273)","(99.0, 2.0086536056)","(99.0, 0.933889068082)","(99.0, 0.960180716926)","(99.0, 1.30440093616)","(-5.63, 1.46025778512)","(99.0, 0.657304218251)","(99.0, 0.0701116458692)","(99.0, 1.12468481597)"
1,"(100.0, 96.3249977745)","(4.08, 1.08879446842)","(-0.29, 0.244614764285)","(6.36, 0.410967929289)","(4.37, -1.59008757301)","(-2.38, 0.617355101917)","(-9.66, 1.92139148451)","(-0.73, -0.518936144518)","(-5.34, -0.474277751105)","(8.88, -0.588065508898)",...,"(2.82, 2.14592980288)","(-4.95, 1.30567385403)","(-0.29, 2.63501141669)","(7.86, 1.34846481634)","(-0.19, 1.08382224897)","(-2.14, 1.66437345885)","(3.06, 1.80896420047)","(0.34, 0.810630583076)","(-4.32, 0.0447716274221)","(1.07, 1.38526638435)"
2,"(49.0, 54.5213464809)","(99.0, 0.701182613059)","(99.0, 0.310254074611)","(99.0, 0.256350059247)","(99.0, -0.692387760974)","(9.03, 0.277155611099)","(9.27, 1.09294537351)","(9.03, -0.176167063512)","(9.27, -0.229936871217)","(99.0, -0.181311949186)",...,"(99.0, 1.18881986886)","(99.0, 0.75193926626)","(99.0, 1.4264558469)","(9.08, 0.562756342986)","(99.0, 0.825003160177)","(99.0, 0.964417520844)","(99.0, 1.12384806675)","(99.0, 0.50883756131)","(99.0, 0.088226200352)","(99.0, 0.872090017229)"
3,"(48.0, 50.0073109714)","(99.0, 0.553647909954)","(8.35, 0.10351791547)","(99.0, 0.210111516256)","(99.0, -0.853866648203)","(1.8, 0.330376846488)","(8.16, 0.996754943785)","(-2.82, -0.285469720445)","(6.21, -0.25148434134)","(99.0, -0.326001801215)",...,"(99.0, 1.11758996004)","(99.0, 0.676079278028)","(99.0, 1.37685268279)","(0.53, 0.727453410587)","(99.0, 0.533763356268)","(99.0, 0.861007771107)","(99.0, 0.925470535519)","(99.0, 0.414007307438)","(99.0, 0.0146507848959)","(99.0, 0.707138305516)"
4,"(91.0, 93.8099811418)","(8.5, 1.13889147692)","(4.61, 0.397109542008)","(-4.17, 0.422189277558)","(-5.39, -1.35655662387)","(1.36, 0.534394033104)","(1.6, 1.87622769144)","(7.04, -0.396667380272)","(4.61, -0.426278816765)","(-0.44, -0.432563556257)",...,"(5.19, 2.06603306915)","(5.58, 1.28352144285)","(4.27, 2.50610026402)","(5.19, 1.12783700874)","(5.73, 1.25116271204)","(1.55, 1.64159413753)","(3.11, 1.85416600133)","(6.55, 0.835714146656)","(1.8, 0.101759359923)","(1.6, 1.43048951961)"
5,"(100.0, 91.632125159)","(-6.17, 1.02759677507)","(-3.54, 0.21620211195)","(0.44, 0.388666884376)","(-8.5, -1.53255493617)","(-7.09, 0.594217689439)","(-4.32, 1.82726348022)","(-8.69, -0.504941367631)","(-0.87, -0.454868991127)","(-6.65, -0.573965715475)",...,"(-3.54, 2.04385982254)","(-6.89, 1.24082307664)","(-0.68, 2.51287674387)","(-2.96, 1.30201919531)","(-2.18, 1.01070795741)","(-3.35, 1.58113988727)","(0.05, 1.71123648313)","(-9.08, 0.766335745158)","(-5.05, 0.0365525109633)","(-3.45, 1.30932718335)"
6,"(47.0, 50.2714002238)","(99.0, 0.572524432902)","(99.0, 0.136342051469)","(99.0, 0.21568067604)","(99.0, -0.819367437456)","(8.59, 0.318542317685)","(-9.85, 1.003035232)","(7.72, -0.264890471748)","(8.79, -0.245576958856)","(99.0, -0.299251825046)",...,"(99.0, 1.11864325545)","(99.0, 0.682074984636)","(99.0, 1.37191189757)","(99.0, 0.69362629867)","(99.0, 0.576326989678)","(2.33, 0.869755376136)","(99.0, 0.949136168561)","(99.0, 0.425589338297)","(99.0, 0.0265429324895)","(99.0, 0.727407839738)"
7,"(100.0, 95.9404561856)","(6.84, 1.13731273058)","(3.16, 0.350600898062)","(9.17, 0.424105602511)","(-6.21, -1.45447143139)","(-8.16, 0.569890926673)","(-1.7, 1.91708928563)","(9.27, -0.443672072049)","(1.41, -0.448407093089)","(-5.19, -0.491367263509)",...,"(7.23, 2.12129507401)","(-1.12, 1.30849859742)","(-0.1, 2.58402350073)","(-5.68, 1.21825279169)","(-3.16, 1.21120389408)","(-3.35, 1.671649269

In [11]:
np.amax(predictions)

145.10523622936088

In [12]:
np.amin(predictions)

-6.2754125172197011

In [ ]:
d = comparison_data.to_latex()
text_file = open("comparison.txt", "w")
text_file.write(d)
text_file.close()